In [1]:
import pandas as pd
import numpy as np

import os
import os.path
import re
import string
import collections
import math
from collections import Counter
import random

frequentwords = ['a', 'all', 'also', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', 'do', 'down', 'even', 'every', 'for', 'from', 'had', 'has', 'have', 'her', 'his', 'if', 'in', 'into', 'is', 'it', 'its', 'may', 'more', 'must', 'my', 'no', 'not', 'now', 'of', 'on', 'one', 'only', 'or', 'our', 'shall', 'should', 'so', 'some', 'such', 'than', 'that', 'the', 'their', 'then', 'there', 'things', 'this', 'to', 'up', 'upon', 'was', 'were', 'what', 'when', 'which', 'who', 'will', 'with', 'would', 'your']

path = '../data/Raw/blogs/blogs/'
files = os.listdir(path)
files.sort() 

df = pd.read_csv('../data/Processed/dataframe_2.csv')
df = df.sort_values('total_words', ascending = False)

In [9]:
top_10 = list(df.groupby('unknown').size().sort_values().tail(10).index)

data = df[np.isin(df.unknown,top_10)].drop(['id','age','star_sign','total_words','gender'],axis=1)
data

,unknown,a,all,also,an,and,any,are,as,at,...,were,what,when,which,who,will,with,would,your,other_words
18813,indUnk,8208,1973,377,853,14085,450,1926,2089,2601,...,769,1499,1494,225,489,2064,3675,948,608,264173
19093,Student,10832,1780,349,1416,10053,723,1841,1652,2371,...,771,1556,964,614,1063,1083,2346,821,1446,262154
18966,indUnk,7785,1550,630,742,10061,406,730,1236,1730,...,881,883,778,824,366,459,2249,561,298,215372
176,Education,8461,805,94,797,7596,284,1005,2105,2397,...,344,784,434,257,366,350,2401,371,599,194343
19263,indUnk,5221,1483,387,698,9461,387,1324,2284,1340,...,578,962,935,1408,1389,764,1729,707,429,175904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15156,indUnk,4,1,0,0,3,0,5,1,1,...,0,1,0,0,1,0,1,1,1,120
16976,indUnk,0,0,1,0,0,1,0,0,2,...,0,0,1,0,0,1,0,0,0,76
3635,Student,1,0,0,0,0,1,0,0,2,...,0,1,1,0,0,0,0,0,0,39
3902,Student,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33


In [10]:
X = data.drop(['unknown'],axis=1)
y = data['unknown']
data['unknown'].unique()

array(['indUnk', 'Student', 'Education', 'Non-Profit', 'Arts',
       'Technology', 'Internet', 'Engineering', 'Communications-Media',
       'Government'], dtype=object)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

log_model = LogisticRegression(solver = 'saga',penalty = 'l1',max_iter = 1000)

X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

log_model.fit(X_train,y_train)

C:\Users\jonah\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=1000, penalty='l1', solver='saga')

In [13]:
preds = log_model.predict(X_test)
sum(np.where(preds == y_test,1,0))/len(X_test)

0.42556436851738866

In [14]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(algorithm = 'brute')
knn_model.fit(X_train,y_train)

KNeighborsClassifier(algorithm='brute')

In [15]:
preds = knn_model.predict(X_test)
sum(np.where(preds == y_test,1,0))/len(X_test)

0.3483831604636974

In [16]:
from sklearn.tree import DecisionTreeClassifier 
tree_model = DecisionTreeClassifier(max_depth = 200)

tree_model.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=200)

In [17]:
preds = tree_model.predict(X_test)
sum(np.where(preds == y_test,1,0))/len(X_test)

0.2788285539963392